## Import

In [1]:
from wmb import cemba
from ALLCools.mcds import MCDS

import pandas as pd
from ALLCools.clustering import log_scale
import pandas as pd
import glob
import xarray as xr
import numpy as np

In [ ]:
# Parameters
chrom_to_remove = ["chrX", "chrY", "chrM", "chrL"]
cpu = 1
downsample = 1000
mc_type = "CHN"
mem_gb = 1
std_cutoff = 0.05
var_dim = "chrom100k"


## Select cells

In [3]:
meta = pd.read_csv('/home/qzeng_salk_edu/project/230712_m3c-mc-integration/mC_META_230814.csv', index_col = 0)
use_cells = pd.read_csv('mc_cells.txt', index_col=0, header=None).index
use_cells.name = 'cell'

## Get adata with CEF

In [4]:
mcds_paths = glob.glob('/home/qzeng_salk_edu/project/230712_m3c-mc-integration/mcds_100kb/*/mcds/*.mcds')
len(mcds_paths)

75

In [6]:
features_df = pd.read_csv('/home/qzeng_salk_edu/project/230712_m3c-mc-integration/mc_m3c_chrom100k_id.csv', index_col = 0)
m3c_to_mc = features_df.set_index('m3c_id')['mc_id'].to_dict()

In [7]:
mcds = MCDS.open(mcds_paths, var_dim=var_dim, use_obs=use_cells).sel(chrom100k = features_df['mc_id'].tolist())

mcds.add_mc_frac(
normalize_per_cell=True,  # after calculating mC frac, per cell normalize the matrix
    clip_norm_value=10)

In [8]:
mcds

<xarray.MCDS>
Dimensions:              (mc_type: 2, cell: 12501, geneslop2k: 55487, count_type: 2, chrom100k: 26350)
Coordinates:
  * mc_type              (mc_type) <U3 'CGN' 'CHN'
  * cell                 (cell) <U40 '3F_M_100' ... 'AMB_200604_18mo_3F_4E_2_...
  * geneslop2k           (geneslop2k) <U21 'ENSMUSG00000102693.1' ... 'ENSMUS...
    strand_type          <U4 'both'
  * count_type           (count_type) <U3 'mc' 'cov'
    geneslop2k_chrom     (geneslop2k) <U5 'chr1' 'chr1' 'chr1' ... 'chrM' 'chrM'
    geneslop2k_start     (geneslop2k) int64 3071252 3100015 ... 13288 13355
    geneslop2k_end       (geneslop2k) int64 3076321 3104124 ... 16299 16299
  * chrom100k            (chrom100k) int64 0 1 2 3 4 ... 26346 26347 26348 26349
    chrom100k_chrom      (chrom100k) <U5 'chr1' 'chr1' 'chr1' ... 'chrX' 'chrX'
    chrom100k_bin_start  (chrom100k) int64 0 100000 ... 170900000 171000000
    chrom100k_bin_end    (chrom100k) int64 100000 200000 ... 171000000 171031299
Data variables:
    geneslop2k_da        (cell, geneslop2k, mc_type, count_type) int64 dask.array<chunksize=(253, 55487, 2, 2), meta=np.ndarray>
    chrom100k_da         (cell, chrom100k, mc_type, count_type) int64 dask.array<chunksize=(253, 26350, 2, 2), meta=np.ndarray>
    chrom100k_da_frac    (cell, chrom100k, mc_type) float64 dask.array<chunksize=(253, 26350, 2), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom100k

In [9]:
# new_index = [mc_to_m3c[mc_index] for mc_index in mcds.get_index('chrom100k')]
# mcds = mcds.reindex(chrom100k=new_index)

In [10]:
mc_cef = pd.read_csv('mC.CEF.csv', header=None, index_col=0).squeeze()
rna_cef = pd.read_csv('m3C.CEF.csv', header=None, index_col=0).squeeze()
cef = mc_cef | rna_cef
cef = cef[cef].index
cef.name = var_dim

In [11]:
cef = [m3c_to_mc[m3c_id] for m3c_id in cef]

In [12]:
mcds.add_feature_selection_column(cef, f'{var_dim}_cef')
adata = mcds.get_adata(mc_type=mc_type,
                       select_hvf=f'{var_dim}_cef',
                       split_large_chunks=False)

In [13]:
std_filter = adata.X.std(axis=0) > std_cutoff
adata._inplace_subset_var(std_filter)

In [14]:
adata

AnnData object with n_obs × n_vars = 12501 × 1328
    var: 'chrom', 'bin_start', 'bin_end', 'cef'

## Preprocessing and save

In [15]:
# log mC fraction and scale features
log_scale(adata, with_mean=True)

StandardScaler()

In [20]:
#features_df = pd.read_csv('/home/qzeng_salk_edu/project/230712_m3c-mc-integration/mc_m3c_chrom100k_id.csv', index_col = 0)
features_df['mc_id'] = features_df['mc_id'].astype(str)
mc_to_m3c = features_df.set_index('mc_id')['m3c_id'].to_dict()

In [21]:
adata.var_names = [mc_to_m3c[mc_id] for mc_id in adata.var_names]

In [22]:
adata.write_h5ad('mc_input.h5ad')

In [44]:
# adata.obs['CellType'] = meta['MajorType']

In [45]:
# adata.obs